# Final ML Tuning
## Team A

In [1]:
!pip install tf-keras
!pip install evaluate
!pip install matplotlib
!pip install seaborn
!pip install accelerate>=0.26.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import sqlite3
import evaluate
import os
import json
import matplotlib.pyplot as plt
import seaborn
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)
import torch
from torch.utils.data import Dataset

C:\Users\rza_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def execute_query(query, print_padding, verbose=True):
    try:
        conn = sqlite3.connect("data\\reza_data_cars.db")
        cursor = conn.cursor()
        cursor.execute(query)
        if verbose:
            print(f"{'SQL query:':<{print_padding}}{query}")
            print(f"{'Query results:':<{print_padding}}{cursor.fetchall()}")
        return cursor.fetchall()
    except sqlite3.Error as error:
        print(f"Error executing sql {error}")
        
    finally:
        conn.close()

# Load the tokenizer and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql').to(device)

# fine-tune the model
# t5 fine-tuning article https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887


model.eval()

def generate_sql(input_prompt):
    """Generate SQL query from natural language input."""
    inputs = tokenizer(input_prompt, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

padding = 30
print("Good examples:\n")
# example 
natural_language_query = "How many makers are there?"
input_prompt = f"""tables:
CREATE TABLE "car_makers" ( 
	Id INTEGER PRIMARY KEY, 
	Maker TEXT, 
	FullName TEXT, 
	Country TEXT,
	FOREIGN KEY (Country) REFERENCES countries(CountryId)
)
query for: {natural_language_query}"""

generated_sql = generate_sql(input_prompt)
print(f"{'Original query:':<{padding}}{natural_language_query}\n{'Generated SQL:':<{padding}}{generated_sql}")
execute_query(generated_sql, padding)

# example 
natural_language_query = "Who is the heaviest car?"
input_prompt = f"""tables:
CREATE TABLE "cars_data" (
	Id INTEGER PRIMARY KEY, 
	MPG TEXT, 
	Cylinders INTEGER, 
	Edispl REAL, 
	Horsepower TEXT, 
	Weight INTEGER, 
	Accelerate REAL, 
	Year INTEGER,
	FOREIGN KEY (Id) REFERENCES car_names (MakeId)
)
CREATE TABLE "car_names" ( 
	MakeId INTEGER PRIMARY KEY, 
	Model TEXT, 
	Make TEXT,
	FOREIGN KEY (Model) REFERENCES model_list (Model)
)
query for: {natural_language_query}"""

generated_sql = generate_sql(input_prompt)
print(f"\n{'Original query:':<{padding}}{natural_language_query}\n{'Generated SQL:':<{padding}}{generated_sql}")
execute_query(generated_sql, padding)

# example 
natural_language_query = "How many makers are there?"
input_prompt = f"""tables:
CREATE TABLE "model_list" (
	Model TEXT UNIQUE
)
query for: {natural_language_query}"""

generated_sql = generate_sql(input_prompt)
print(f"\n{'Original query:':<{padding}}{natural_language_query}\n{'Generated SQL:':<{padding}}{generated_sql}")
execute_query(generated_sql, padding)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Good examples:

Original query:               How many makers are there?
Generated SQL:                SELECT COUNT(*) FROM "car_makers"
SQL query:                    SELECT COUNT(*) FROM "car_makers"
Query results:                [(22,)]

Original query:               Who is the heaviest car?
Generated SQL:                SELECT Car_names, MAX(Car_names) FROM "car_names" GROUP BY Car_names ORDER BY MAX(Car_names) DESC LIMIT 1
Error executing sql no such column: Car_names

Original query:               How many makers are there?
Generated SQL:                SELECT COUNT(*) FROM "model_list"
SQL query:                    SELECT COUNT(*) FROM "model_list"
Query results:                [(36,)]


[]

## Improve model with fine-tuning


Training data to fine-tune the model

In [4]:
# Example data to fine-tune the model
training_data = [
    {
        "input": """tables:
CREATE TABLE "car_names" ( 
    MakeId INTEGER PRIMARY KEY, 
    Model TEXT, 
    FOREIGN KEY (Model) REFERENCES model_list (Model)
)
CREATE TABLE "model_list" ( 
    ModelId INTEGER PRIMARY KEY, 
    Maker INTEGER, 
    Model TEXT UNIQUE,
    FOREIGN KEY (Maker) REFERENCES car_makers (Id)
)
CREATE TABLE "car_makers" ( 
    Id INTEGER PRIMARY KEY, 
    Maker TEXT, 
    FullName TEXT
)
query for: List all unique car models along with their maker full names.""",
        "target": """
select DISTINCT(car_names.Model), car_makers.FullName 
from car_names 
join model_list on car_names.Model = model_list.Model 
join car_makers on model_list.Maker = car_makers.Id
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY, 
    Year INTEGER,
    FOREIGN KEY (Id) REFERENCES car_names (MakeId)
)
CREATE TABLE "car_names" ( 
    MakeId INTEGER PRIMARY KEY, 
    Model TEXT,
    FOREIGN KEY (Model) REFERENCES model_list (Model)
)
CREATE TABLE "model_list" (
    ModelId INTEGER PRIMARY KEY,
    Maker INTEGER,
    Model TEXT UNIQUE,
    FOREIGN KEY (Maker) REFERENCES car_makers (Id)
)
CREATE TABLE "car_makers" ( 
    Id INTEGER PRIMARY KEY,
    Country TEXT,
    FOREIGN KEY (Country) REFERENCES countries(CountryId)
)
CREATE TABLE "countries" (
    CountryId INTEGER PRIMARY KEY, 
    CountryName TEXT
)
query for: Show each car's model, year, and country of origin.""",
        "target": """
SELECT car_names.Model, cars_data.Year, countries.CountryName
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
JOIN model_list ON car_names.Model = model_list.Model
JOIN car_makers ON model_list.Maker = car_makers.Id
JOIN countries ON car_makers.Country = countries.CountryId
""",
    },
    {
        "input": """tables:
CREATE TABLE "car_makers" ( 
    Id INTEGER PRIMARY KEY, 
    FullName TEXT, 
    Country TEXT,
    FOREIGN KEY (Country) REFERENCES countries(CountryId)
)
CREATE TABLE "countries" (
    CountryId INTEGER PRIMARY KEY, 
    CountryName TEXT
)
query for: Find all car makers from Japan.""",
        "target": """
SELECT car_makers.FullName
FROM car_makers
JOIN countries ON car_makers.Country = countries.CountryId
WHERE countries.CountryName LIKE 'japan'
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY, 
    Horsepower TEXT,
    FOREIGN KEY (Id) REFERENCES car_names (MakeId)
)
CREATE TABLE "car_names" ( 
    MakeId INTEGER PRIMARY KEY, 
    Model TEXT,
    FOREIGN KEY (Model) REFERENCES model_list (Model)
)
CREATE TABLE "model_list" (
    ModelId INTEGER PRIMARY KEY,
    Maker INTEGER,
    Model TEXT UNIQUE,
    FOREIGN KEY (Maker) REFERENCES car_makers (Id)
)
CREATE TABLE "car_makers" ( 
    Id INTEGER PRIMARY KEY, 
    Country TEXT,
    FOREIGN KEY (Country) REFERENCES countries(CountryId)
)
CREATE TABLE "countries" (
    CountryId INTEGER PRIMARY KEY, 
    CountryName TEXT
)
query for: What is the average horsepower of cars made in Germany?""",
        "target": """
SELECT AVG(CAST(cars_data.Horsepower AS FLOAT)) AS avg_hp
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
JOIN model_list ON car_names.Model = model_list.Model
JOIN car_makers ON model_list.Maker = car_makers.Id
JOIN countries ON car_makers.Country = countries.CountryId
WHERE countries.CountryName LIKE 'Germany'
""",
    },
    {
        "input": """tables:
CREATE TABLE "car_makers" ( 
    Id INTEGER PRIMARY KEY, 
    Country TEXT,
    FOREIGN KEY (Country) REFERENCES countries(CountryId)
)
CREATE TABLE "countries" (
    CountryId INTEGER PRIMARY KEY, 
    Continent INTEGER
)
CREATE TABLE "continents" ( 
    ContId INTEGER PRIMARY KEY, 
    Continent TEXT 
)
query for: Which continent has the most car makers?""",
        "target": """
SELECT continents.Continent, COUNT(*) AS maker_count
FROM car_makers
JOIN countries ON car_makers.Country = countries.CountryId
JOIN continents ON countries.Continent = continents.ContId
GROUP BY continents.Continent
ORDER BY maker_count DESC
LIMIT 1
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    MPG TEXT,
    Year INTEGER
)
query for: What is the average MPG per year?""",
        "target": """
SELECT Year, AVG(CAST(MPG AS FLOAT)) AS avg_mpg
FROM cars_data
GROUP BY Year
ORDER BY Year
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    Cylinders INTEGER
)
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Model TEXT
)
query for: What car models have more than 6 cylinders?""",
        "target": """
SELECT car_names.Model
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
WHERE cars_data.Cylinders > 6
""",
    },
    {
        "input": """tables:
CREATE TABLE "car_makers" (
    Id INTEGER PRIMARY KEY,
    Country TEXT
)
CREATE TABLE "countries" (
    CountryId INTEGER PRIMARY KEY,
    Continent INTEGER
)
CREATE TABLE "continents" (
    ContId INTEGER PRIMARY KEY,
    Continent TEXT
)
query for: How many car makers are there in each continent?""",
        "target": """
SELECT continents.Continent, COUNT(*) AS maker_count
FROM car_makers
JOIN countries ON car_makers.Country = countries.CountryId
JOIN continents ON countries.Continent = continents.ContId
GROUP BY continents.Continent
ORDER BY maker_count DESC
""",
    },
    {
        "input": """tables:
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Model TEXT
)
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    Weight INTEGER
)
CREATE TABLE "model_list" (
    ModelId INTEGER PRIMARY KEY,
    Maker INTEGER,
    Model TEXT UNIQUE
)
CREATE TABLE "car_makers" (
    Id INTEGER PRIMARY KEY,
    FullName TEXT
)
query for: What is the lightest car made by each maker?""",
        "target": """
SELECT car_makers.FullName, MIN(cars_data.Weight) AS lightest
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
JOIN model_list ON car_names.Model = model_list.Model
JOIN car_makers ON model_list.Maker = car_makers.Id
GROUP BY car_makers.FullName
ORDER BY lightest ASC
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    Year INTEGER
)
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Model TEXT
)
query for: Which car models were released in the 1970s?""",
        "target": """
SELECT car_names.Model
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
WHERE Year BETWEEN 1970 AND 1979
""",
    },
    {
        "input": """tables:
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Make TEXT
)
query for: Find all car models that contain the word 'Civic'.""",
        "target": """
SELECT Make
FROM car_names
WHERE Make LIKE '%Civic%'
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    Accelerate REAL
)
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Model TEXT
)
query for: What are the top 3 fastest accelerating cars?""",
        "target": """
SELECT car_names.Model, cars_data.Accelerate
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
ORDER BY cars_data.Accelerate ASC
LIMIT 3
""",
    },
    {
        "input": """tables:
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Model TEXT
)
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    Weight INTEGER
)
CREATE TABLE "model_list" (
    ModelId INTEGER PRIMARY KEY,
    Maker INTEGER,
    Model TEXT
)
CREATE TABLE "car_makers" (
    Id INTEGER PRIMARY KEY,
    FullName TEXT
)
query for: Which makers have built a car lighter than 2000 pounds?""",
        "target": """
SELECT DISTINCT car_makers.FullName
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
JOIN model_list ON car_names.Model = model_list.Model
JOIN car_makers ON model_list.Maker = car_makers.Id
WHERE cars_data.Weight < 2000
""",
    },
    {
        "input": """tables:
CREATE TABLE "cars_data" (
    Id INTEGER PRIMARY KEY,
    Year INTEGER
)
CREATE TABLE "car_names" (
    MakeId INTEGER PRIMARY KEY,
    Model TEXT
)
query for: Show me all car models not made in the 1980s.""",
        "target": """
SELECT car_names.Model
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
WHERE Year < 1980 OR Year > 1989
""",
    },
]

In [5]:
# re-pull model to ensure I'm running against the right model in noteboook
# Use pre-trained model to test output of training data before fine-tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql').to(device)

match_count = 0
for example in training_data:
    input_prompt = example["input"]
    expected_sql = example["target"]
    generated_sql = generate_sql(input_prompt)
    generated_results = execute_query(generated_sql, padding, verbose=False)
    print(f"\n{'Generated SQL:':<{padding}}{generated_results}")
    expected_sql_results = execute_query(expected_sql, padding, verbose=False)
    print(f"{'Expected SQL:':<{padding}}{expected_sql_results}")
    print("**" * 50)
    if generated_results == expected_sql_results:
        match_count += 1

print(f"Total Matches: {match_count} out of {len(training_data)}")

Error executing sql no such column: Model

Generated SQL:                None
Expected SQL:                 [('chevrolet', 'General Motors'), ('buick', 'General Motors'), ('plymouth', 'Chrysler'), ('amc', 'American Motor Company'), ('ford', 'Ford Motor Company'), ('pontiac', 'General Motors'), ('citroen', 'Citroen'), ('dodge', 'Chrysler'), ('toyota', 'Toyota'), ('datsun', 'Nissan Motors'), ('volkswagen', 'Volkswagen'), ('peugeot', 'Peugeaut'), ('audi', 'Volkswagen'), ('saab', 'Saab'), ('bmw', 'BMW'), ('mercury', 'Ford Motor Company'), ('opel', 'Opel'), ('fiat', 'Fiat'), ('oldsmobile', 'General Motors'), ('chrysler', 'Chrysler'), ('mazda', 'Mazda'), ('volvo', 'Volvo'), ('renault', 'Renault'), ('honda', 'Honda'), ('subaru', 'Subaru'), ('capri', 'Ford Motor Company'), ('mercedes-benz', 'Daimler Benz'), ('cadillac', 'General Motors'), ('mercedes', 'Daimler Benz'), ('triumph', 'Triumph'), ('nissan', 'Nissan Motors')]
**************************************************************************

In [ ]:
class TextToSQLDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data[idx]['input']
        target_text = self.data[idx]['target']
        
        # Tokenize inputs and targets
        inputs = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        targets = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        
        return {
            'input_ids': inputs.input_ids.squeeze(0),
            'attention_mask': inputs.attention_mask.squeeze(0),
            'labels': targets.input_ids.squeeze(0)
        }

In [7]:
# re-pull model to ensure I'm running against the right model in noteboook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql').to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50, # Will be overfitting the model
    per_device_train_batch_size=1,
    learning_rate=5e-4,
    weight_decay=0.0,
    logging_dir='./logs',
    # logging_steps=1, # commented out for smaller export
    save_strategy="no",
    remove_unused_columns=False,
    report_to="none",
)

dataset = TextToSQLDataset(tokenizer, training_data, max_length=256)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')

C:\Users\rza_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\data\data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.040100


('./fine-tuned-model\\tokenizer_config.json',
 './fine-tuned-model\\special_tokens_map.json',
 './fine-tuned-model\\spiece.model',
 './fine-tuned-model\\added_tokens.json')

In [8]:
# Use the newly fine-tuned model to generate SQL queries
# I am using the same questions as what was provided in the fine-tuning data
# to see if the fine-tuned model can generate the correct SQL queries after the training.
fine_tuned_model_path = './fine-tuned-model'
tokenizer = T5Tokenizer.from_pretrained(fine_tuned_model_path)
model = T5ForConditionalGeneration.from_pretrained(fine_tuned_model_path).to(device)

def generate_sql_fine_tuned(input_prompt):
    """Generate SQL query from natural language input using the fine-tuned model."""
    inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated SQL query's results and compare them with the expected query's results from training
match_count = 0
for example in training_data:
    input_prompt = example["input"]
    expected_sql = example["target"]
    generated_sql = generate_sql_fine_tuned(input_prompt)
    generated_results = execute_query(generated_sql, padding, verbose=False)
    expected_sql_results = execute_query(expected_sql, padding, verbose=False)
    if generated_results == expected_sql_results:
        match_count += 1
    else:
        print(f"\n{'Generated SQL:':<{padding}}{generated_sql}")
        print(f"{'Expected SQL:':<{padding}}{expected_sql}")

print(f"Total Matches: {match_count} out of {len(training_data)}")

Error executing sql near "2000": syntax error

Generated SQL:                SELECT DISTINCT car_makers.FullName FROM cars_data JOIN car_names ON cars_data.Id = car_names.MakeId JOIN model_list ON car_names.Model = model_list.Model JOIN car_makers ON model_list.Maker = car_makers.Id WHERE cars_data.Weight  2000
Expected SQL:                 
SELECT DISTINCT car_makers.FullName
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
JOIN model_list ON car_names.Model = model_list.Model
JOIN car_makers ON model_list.Maker = car_makers.Id
WHERE cars_data.Weight < 2000

Error executing sql near "1980": syntax error

Generated SQL:                SELECT car_names.Model FROM cars_data JOIN car_names ON cars_data.Id = car_names.MakeId WHERE Year  1980 OR Year > 1989
Expected SQL:                 
SELECT car_names.Model
FROM cars_data
JOIN car_names ON cars_data.Id = car_names.MakeId
WHERE Year < 1980 OR Year > 1989

Total Matches: 12 out of 14


# Why does it still NOT understand `<` and `>` :(

In [10]:
# ************************************************************** #
# Queries that weren't part of the training data
input_prompt = """tables:
CREATE TABLE "model_list" ( 
	ModelId INTEGER PRIMARY KEY, 
	Maker INTEGER, 
	Model TEXT UNIQUE
)
CREATE TABLE "car_names" ( 
	MakeId INTEGER PRIMARY KEY, 
	Model TEXT, 
	Make TEXT,
	FOREIGN KEY (Model) REFERENCES model_list (Model)
)
query for: how many cars did BMW make in 1980?"""
generated_sql = generate_sql_fine_tuned(input_prompt)
print(f"Original query: {natural_language_query}\nGenerated SQL: {generated_sql}")
execute_query(generated_sql, padding)


Original query: How many makers are there?
Generated SQL: SELECT car_names.Model, cars FROM cars_list JOIN car_names ON cars_list.Make = car_names.Model JOIN model_list ON car_names.Make = car_names.MakeId WHERE YEAR = 1980 OR YEAR = 1980
Error executing sql no such table: cars_list
